### FFT Average Project
This project uses two ADC input running at 3.072 GHz as inputs to a 16 channels polyphase filter bank followed by 16 FFTs, 32768 points each. The speed of ADCs could be further increased to the maximum of this FPGA, which is 4 GSPS. Together, both ADCs allow to cover up to 4 GHz of analog bandwidth.

#### Input ADCs and mapping
The project was developed using ZCU111 and its companion XM500 extension board. Inputs are taken from ADC 224 CH0 for I, and ADC 224 CH1 for Q. These inputs are coupled by Low-Frequency baluns. The project can be easily recompiled using different ADC channels for I and Q.

#### Polyphase Filter Bank
ADC outputs use a AXI Stream interface to deliver the digital samples. Due to the fast speed of the converters, each AXI stream clock gives 8 parallel samples from the ADC. The polyphase filter bank is architectured as a multi-lane approach to take advantage of these parallelized samples. The PFB is 16 channels, 50 % overlap between the channels. This overlap allows to ensure a flat reponse for the PFB over the entire bandwidth.

I/Q samples coming from ADCs are fed into 16 Xilinx FIR IPs, which implement the Polyphase Decomposition of the base filter needed for the PFB. The output of those filters is combined and sorted to be presented to the first FFT, also part of the PFB implementation. This FFT is based on a SytemGenerator created 16-point FFT, with 16 parallel inputs. It means each clock cycle the SSR FFT takes 16 inputs and creates 16 output samples. Each output sample represent one sample of a time-domain signal.

Channel center is given by K\*fs_adc/16, where K is the channel number. The bandwidth of the recovered channels is fs_adc/8. This reduction by 8 instead of 16 makes it possible to have half the spectrum of the recovered channels to be overlapped with their neighbors.

In the overlap PFB implementation, odd channels need to be frequency shifted by pi, which corresponds to multiply the time samples by +1 and -1. This is done by the axis pimod block, which comes right after the SSR FFT.

#### FFT for spectrum of individual channels
The project is not finished yet. In the future, the project will be completed by performing few 100s of averages of the spectrum of the recovered channels. The actual implementation is used as a demonstrator. For this reason, the block axis xfft 16x32768 integrates 16 Xilinx FFT blocks, 32768 points each. This block eats a nice amount of resources of the FPGA, but together with the previous PFB allows to compute the spectrum of the 4 GHz signal without any gaps in the time domain, with a frequency resolution of about 15 kHz.

#### Channel selection and buffering
In this version of the Firmware, a block was added to extract only one channel and decrease the size of the output buffer before transferring data into the Pynq environment. The channel selection block receives all 16 FFT outputs and routes whatever channel is selected into the output. The buffer at the end of the chain is implemented using URAM memory. This allows to let most of the BRAM available for FFT blocks.

#### Output DAC
To allow a self contained loop-back project, a DAC output was added that can be used to inject a single tone into the PFB structure. DAC 229 CH3 (LF balun) is configured with a frequency of 6.144 GHz and to avoid using any buffering or extra logic to drive it, a complex constant block was developed. This constant block can be easily configured with a constant complex value. This gives a DC input into the DAC. Then, the DAC is used in Mixer Mode, where x2 interpolation and mixing is performed in the core. By controlling the NCO frequency a single output tone can be created with an extremely cheap piece of logic, due to the fact that the NCO and mixer are integrated into the DAC core.

#### Examples to jump start
Two simple examples are provided. The first example creates a tone using DAC 229 CH3 as output, and loops-back into the ADC 224 CH0 (only I is provided). ADC 224 CH1 can be 50 Ohm terminated. Depending on the output frequency, the recovered signal will show up on different channels.

The second example is intended to understand the averaging of the spectrum of channels. In this example, an external noise source is recommended to provide wide-band white noise into the system. The example will select one particular channel and perform spectrum averaging by capturing multiple fft outputs and performing the averaging in Python. The averaging process should lower the variance of the noise.

In [ ]:
import os
from pynq import Overlay
import xrfclk
import xrfdc
import numpy as np
import scipy as sp
import scipy.signal
from pynq import Overlay, allocate
from pynq.lib import AxiGPIO
import matplotlib.pyplot as plt
import time
import scipy.io as sio
from scipy import fft, ifft

In [ ]:
#########################
### Support functions ###
#########################

# Format buffer: 
# Lower 16 bits: I
# Next 16 bits: Q
# Upper 16 bits: K index of FFT.
def format_buffer(buff):
    # Format: 
    data = buff
    dataI = data & 0xFFFF
    dataI = dataI.astype(np.int16)
    dataQ = (data >> 16) & 0xFFFF
    dataQ = dataQ.astype(np.int16)
    index = (data >> 48) & 0xFFFF
    index = index.astype(np.uint16)
    
    return dataI,dataQ,index

# Bit reversal: n is the number of bits, i.e., log2(FFT SIZE)
def bit_reverse (i, n):
    return int(format(i, '0%db' % n)[::-1], 2)

# Sort FFT data. Output FFT is bit-reversed. Index is given by idx array.
def sort_br(y, idx):
    y_sort = np.zeros(len(y)) + 1j*np.zeros(len(y))
    for i in np.arange(len(y)):
        y_sort[idx[i]] = y[i]
        
    return y_sort

In [ ]:
class SocIp:
    REGISTERS = {}    
    
    def __init__(self, ip, **kwargs):
        self.ip = ip
        
    def write(self, offset, s):
        self.ip.write(offset, s)
        
    def read(self, offset):
        return self.ip.read(offset)
    
    def __setattr__(self, a ,v):
        if a in self.__class__.REGISTERS:
            self.ip.write(4*self.__class__.REGISTERS[a], v)
        else:
            return super().__setattr__(a,v)
    
    def __getattr__(self, a):
        if a in self.__class__.REGISTERS:
            return self.ip.read(4*self.__class__.REGISTERS[a])
        else:
            return super().__getattr__(a)
        
class AxisSsrFft16x16(SocIp):
    # AXIS SSR FFT 16 inputs, 16 points registers.
    # SCALE_REG : scale stages (1 bit per stage. Log2(NFFT) stages).
    # * 0 : don't scale stage.
    # * 1 : scale stage.
    #
    # QOUT_REG : output quantization selection. Register indicates MSB index.
    #
    # ROUND_MODE_REG : output rounding type:
    # * 0 : truncation.
    # * 1 : round.
    #
    REGISTERS = {'scale' : 0, 'qout' : 1, 'round_mode' : 2}
    
    # Generics.
    NFFT = 16
    SSR = 16
    B = 16
    
    def __init__(self, ip):
        # Initialize ip
        super().__init__(ip)
        
        # Default registers:
        # -> no scaling
        # -> q = 3 (maximize dynamic range)
        # -> r = 1 (round)
        self.scale = 0
        self.qout = 3
        self.round_mode = 1
    
    def config(self, s = "yes", q = 0, r = 0):
        # Scale.
        if s == "yes":
            self.scale = int(np.log2(self.NFFT)-1)
        elif s == "no":
            self.scale = 0
        else:
            print("Not a valid scale option")

        # Quantization.
        self.qout = q
        
        # Rounding mode.
        self.round_mode = r
              
class AxisXfft16x32768(SocIp):
    # AXIS XFFT, 16x32768 points. Registers:
    # SCALE_REG : scale individual FFT sections.
    # * 0 : don't scale section.
    # * 1 : scale section.
    #
    # WE_REG
    # * 0 :
    # * 1 : exec.
    #
    # NOTE: the same configuration is applied to all 16 xfft blocks.
    REGISTERS = {'scale' : 0, 'we' : 1}
    
    # FFT size.
    N = 32768
    
    def __init__(self, ip):
        # Initialize ip
        super().__init__(ip)
        
        # Registers.
        self.scale = 0
        self.we = 0

    def config(self,scale):       
        # Change scale value.
        self.scale = scale
        
        # Configure block.
        self.we = 1
        time.sleep(0.1)
        self.we = 0

class AxisChSelPfbx1(SocIp):
    # AXIS Channel Selection PFB Registers
    # CHID_REG
    REGISTERS = {'chid' : 0}
    
    def __init__(self, ip):
        # Initialize ip
        super().__init__(ip)

    def chsel(self,ch):
        # Change channel
        self.chid = ch         

class AxisBufferUram(SocIp):
    # AXIS_buffer URAM registers.
    # RW_REG
    # * 0 : read operation.
    # * 1 : write operation.
    #
    # START_REG
    # * 0 : stop.
    # * 1 : start operation.
    #
    # SYNC_REG
    # * 0 : don't sync with Tlast.
    # * 1 : sync capture with Tlast.
    #
    # The block will either capture or send data out based on RW_REG operation.
    # Read/write operations will use the entire buffer. Tlast is created at the
    # end of the read to ensure DMA does not hang. Both s_axis_tdata and tuser
    # are captured. Output is always 64 bits, with the lower B bits being the
    # data and the upper 16 tuser. Un-used bits should be zero.
    #
    # With SYNC_REG, the user can control to start the capture after a Tlast
    # has been received at the input interface. Previous samples are discarded,
    # included the one with the Tlast flag, and the capture starts right after
    # that sample. If SYNC_REG is set to 0, the block will start capturing data
    # without waiting for Tlast to happen.
    REGISTERS = {'rw_reg' : 0, 'start_reg' : 1, 'sync_reg' : 2}
    
    # Generics
    N = 16
    B = 32
    
    # Buffer length
    BUFFER_LENGTH = (1 << N)
    
    def __init__(self, ip, axi_dma):
        # Initialize ip
        super().__init__(ip)
        self.dma = axi_dma
        
        # Default registers.
        # Write operation, stopped, sync with Tlast.
        self.rw_reg = 1
        self.start_reg = 0
        self.sync_reg = 1
    
    def capture(self):
        # Enable write operation.
        self.rw_reg = 1
        self.start_reg = 1
        
        # Wait for capture
        time.sleep(0.1)
        
        # Stop capture
        self.start_reg = 0
        
    def transfer(self):
        # Enable read operation.
        self.rw_reg = 0        
        
        # Define buffer:         
        buff = allocate(shape=(self.BUFFER_LENGTH,), dtype=np.uint64)

        # Start transfer.
        self.start_reg = 1

        # DMA data.
        self.dma.recvchannel.transfer(buff)
        self.dma.recvchannel.wait()

        # Stop transfer.
        self.start_reg = 0
        
        # Return data
        return buff
    
    def length(self):
        return (1 << self.N)
    
class AxisConstantIQ(SocIp):
    # AXIS Constant IQ registers:
    # REAL_REG : 16-bit.
    # IMAG_REG : 16-bit.
    # WE_REG   : 1-bit. Update registers.
    REGISTERS = {'real_reg':0, 'imag_reg':1, 'we_reg':2}
    
    # Number of bits.
    B = 16
    MAX_V = 2**(B-1)-1
        
    def __init__(self, ip):
        # Initialize ip
        super().__init__(ip)
        
        # Default registers.
        self.real_reg = 30000
        self.imag_reg = 30000
        
        # Register update.
        self.update()

    def update(self):
        self.we_reg = 1
        time.sleep(1)
        self.we_reg = 0
        
    def set_iq(self,i=1,q=1):
        # Set registers.
        self.real_reg = int(i*self.MAX_V)
        self.imag_reg = int(q*self.MAX_V)
        
        # Register update.
        self.update()
        
class Mixer:
    # Get Mixer Object.
    rf=soc.usp_rf_data_converter_0
    dac_tile = rf.dac_tiles[1] # DAC 228: 0, DAC 229: 1
    dac_block = dac_tile.blocks[3] # 0, 1, 2, 3
    dac_mixer = dac_block.MixerSettings

    # Update event.
    event = xrfdc.EVNT_SRC_TILE

    def get_mixer(self):
        return self.dac_block.MixerSettings
    
    def set_freq(self,f):
        # Get actual mixer settings.
        dac_mixer = self.dac_block.MixerSettings
        
        # Make a copy of mixer settings dict
        new_mixcfg = dac_mixer.copy()

        # Update the copy
        new_mixcfg.update({
            'EventSource': self.event,
            'Freq' : f,
            'PhaseOffset' : 0
        })

        # Update settings.
        self.dac_block.MixerSettings = new_mixcfg
        self.dac_block.UpdateEvent(event)

class PfbSoc(Overlay):
    FREF_PLL = 204.8
    fs_adc = 8*384
    fs_dac = 16*384
    
    # Constructor.
    def __init__(self, bitfile, init_clks=False, **kwargs):
        # Load bitstream.
        super().__init__(bitfile, **kwargs)
        
        # Configure PLLs if requested.
        if init_clks:
            self.set_all_clks()        
        
        # Create IP objects.        
        self.ssrfft = AxisSsrFft16x16(self.axis_ssrfft_16x16_0)
        self.fft    = AxisXfft16x32768(self.axis_xfft_16x32768_0)
        self.chsel  = AxisChSelPfbx1(self.axis_chsel_pfb_x1_0)
        self.buf    = AxisBufferUram(self.axis_buffer_uram_0, self.axi_dma_1)
        self.const  = AxisConstantIQ(self.axis_constant_iq_0)
        self.mixer  = Mixer()
                        
    def set_all_clks(self):
        xrfclk.set_all_ref_clks(self.__class__.FREF_PLL)        

In [ ]:
# Load bitstream with custom overlay
soc = PfbSoc('../fft_avg_ad_da.bit', ignore_version=True, init_clks=False)

In [ ]:
#############################
### Simple Sine Loop-back ###
#############################
# Output sine wave (DAC229 CH3, LF balun)
soc.const.set_iq(i=0.95,q=0)
soc.mixer.set_freq(f=260)

# Configure SSRFFT (round)
soc.ssrfft.config(s = "yes", q=3, r = 1)

# Configure FFT
soc.fft.config(scale=0x3f)

# Select channel
K = 1
soc.chsel.chsel(K)
time.sleep(0.5)

# Center frequency.
cf = K*soc.fs_adc/16
if cf>soc.fs_adc/2:
    cf = cf - soc.fs_adc
        
# Capture data.
soc.buf.capture()
time.sleep(0.5)

# Transfer data and format.
buff = soc.buf.transfer()
[yi,yq,idx] = format_buffer(buff)
y = yi + 1j*yq
y=y[0:soc.fft.N]

# Sort data to account for bit-reversal FFT output.
y = sort_br(y,idx)

# Plot spectrum
fs = soc.fs_adc
fs_d = fs/8

YF = np.fft.fftshift(y)
F = np.linspace(-fs_d/2,fs_d/2,len(YF))

plt.figure(1,dpi=150)
plt.plot(F+cf,(20*np.log10((np.abs(YF)/np.max(np.abs(YF))))))
plt.xlabel("f [MHz]");
plt.ylabel("Gain [dB]");
plt.title("Spectrum Single Shot")

In [ ]:
##############################
### FFT Average with Noise ###
##############################
fs = soc.fs_adc
fs_d = fs/8

# Configure SSRFFT (round)
soc.ssrfft.config(s = "no", q=0, r = 0)

# Configure FFT
soc.fft.config(scale=0)

# Select channel
K = 2
soc.chsel.chsel(K)
time.sleep(0.5)

# Average spectrum.
Navg = 30
YF_abs = np.zeros(soc.fft.N)

for i in np.arange(Navg):
    print("Iteration: %d" % i)
    # Capture data.
    soc.buf.capture()
    time.sleep(0.1)

    # Transfer data and format.
    buff = soc.buf.transfer()
    [yi,yq,idx] = format_buffer(buff)
    y = yi + 1j*yq
    y = y[0:soc.fft.N]

    # Sort data to account for bit-reversal FFT output.
    y = sort_br(y,idx)

    # Average.
    YF_abs = YF_abs + np.abs(y)

# Divide by Navg.
YF_abs = YF_abs/Navg

# Plot results.
YF_abs = np.fft.fftshift(YF_abs)
F = np.linspace(-fs_d/2,fs_d/2,len(YF_abs))

# Abs.
plt.figure(1,dpi=150)
plt.plot(F,20*np.log10(YF_abs/np.max(YF_abs)))    
plt.xlabel("f [MHz]")
plt.ylabel("Gain [dB]")
plt.title("Spectrum AVG = 30")
plt.ylim(-80,10)

# Save figure.
plt.figure(1)
fn = "spectrum_" + str(Navg) + ".jpg"
plt.savefig(fn)
